# Using Upstage Document Parse on SageMaker Jumpstart

Upstage Documnet Parse is a powerful API designed to automatically convert any document to HTML. It detects layout elements such as paragraphs, tables, images, and more to determine the structure of the document. The API then serializes the elements according to reading order, and finally converts the document into HTML.

This sample notebook shows you how to deploy [Upstage Document Parse](https://aws.amazon.com/marketplace/pp/prodview-lv5bnpdco7xoq) using Amazon SageMaker.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
2. Ensure that IAM role used has **AmazonSageMakerFullAccess**
3. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        2. **aws-marketplace:Unsubscribe**
        3. **aws-marketplace:Subscribe**  

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Prepare input payload](#B.-Prepare-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
3. [Clean-up](#4.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   2. [Delete the model](#B.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open [Upstage Document Parse](https://aws.amazon.com/marketplace/pp/prodview-lv5bnpdco7xoq) model package listing page.
2. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## IMPORTANT: Setup Environment Variables

Before running this notebook, you need to create a `.env` file in the parent directory (`../`) with your AWS credentials and configuration.

### Steps:
1. Copy the `.env.example` file to create your own `.env` file:
   ```bash
   cp ../.env.example ../.env
   ```

2. Edit the `.env` file and fill in your actual values:
   ```
   # AWS Configuration
   AWS_REGION=us-west-2
   AWS_DEFAULT_REGION=us-west-2
   AWS_ACCOUNT_ID=your-account-id

   # SageMaker Configuration
   SAGEMAKER_EXECUTION_ROLE_ARN=arn:aws:iam::YOUR_ACCOUNT_ID:role/service-role/AmazonSageMaker-ExecutionRole-XXXXXXXX
   DOCUMENT_PARSE_ENDPOINT_NAME=Upstage-Document-Parse-Test
   ```

3. **Required variables:**
   - `SAGEMAKER_EXECUTION_ROLE_ARN`: Your SageMaker execution role ARN
   - `DOCUMENT_PARSE_ENDPOINT_NAME`: Name for your endpoint

The `.env` file is git-ignored, so your credentials will remain secure.

In [ ]:
import os
from dotenv import load_dotenv
import boto3
import sagemaker
from sagemaker import ModelPackage

# Load environment variables from .env file
load_dotenv()

# Verify required environment variables are set
required_vars = ['SAGEMAKER_EXECUTION_ROLE_ARN', 'DOCUMENT_PARSE_ENDPOINT_NAME']
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    raise ValueError(f"Missing required environment variables: {', '.join(missing_vars)}")

# Display loaded environment variables
print("=== Environment Variables ===")
print(f"AWS_REGION: {os.getenv('AWS_REGION')}")
print(f"AWS_DEFAULT_REGION: {os.getenv('AWS_DEFAULT_REGION')}")
print(f"AWS_ACCOUNT_ID: {os.getenv('AWS_ACCOUNT_ID')}")
print(f"SAGEMAKER_EXECUTION_ROLE_ARN: {os.getenv('SAGEMAKER_EXECUTION_ROLE_ARN')}")
print(f"DOCUMENT_PARSE_ENDPOINT_NAME: {os.getenv('DOCUMENT_PARSE_ENDPOINT_NAME')}")
print("=============================\n")

# Load role from environment variable
role = os.getenv('SAGEMAKER_EXECUTION_ROLE_ARN')

sagemaker_session = sagemaker.Session()
sagemaker_runtime = boto3.client("sagemaker-runtime")

In [ ]:
model_package_name = "upstage-document-parse-250404--d656919d867339d6ab3a2b645182a28b"
# if you want to use ml.g5 instances, use below package
# model_package_name = "upstage-document-parse-250404--b6bdea6dc54636a9b2578bd32ac7393f"

# Mapping for Model Packages
model_package_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:model-package/{model_package_name}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{model_package_name}",
    "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{model_package_name}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{model_package_name}",
    "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{model_package_name}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{model_package_name}",
    "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{model_package_name}",
    "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{model_package_name}",
    "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{model_package_name}",
    "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{model_package_name}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{model_package_name}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{model_package_name}",
    "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{model_package_name}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{model_package_name}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{model_package_name}",
    "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{model_package_name}",
}

region = sagemaker_session.boto_region_name
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = model_package_map[region]
print(f"Model Package: '{model_package_arn}'")

region = sagemaker_session.boto_region_name
if region not in model_package_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

model_package_arn = (model_package_map[region])
print(f"Model Package: '{model_package_arn}'")

Model Package: 'arn:aws:sagemaker:us-west-2:594846645681:model-package/upstage-document-parse-250404--d656919d867339d6ab3a2b645182a28b'
Model Package: 'arn:aws:sagemaker:us-west-2:594846645681:model-package/upstage-document-parse-250404--d656919d867339d6ab3a2b645182a28b'


## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
# Load endpoint name from environment variables
endpoint_name = os.getenv('DOCUMENT_PARSE_ENDPOINT_NAME')
real_time_inference_instance_type = (
    "ml.g6.xlarge"
    # if you want to use ml.g5 instances, use `upstage-document-parse-cuda11--a2f49bd4aea13c7b94ba3c034aa524a7` package.
    # "ml.g5.xlarge"
)

print(f"Endpoint Name: {endpoint_name}")
print(f"Instance Type: {real_time_inference_instance_type}")

### A. Create an endpoint

In [ ]:
# Create a deployable model from the model package
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

In [8]:
# Deploy the model
model.deploy(1, real_time_inference_instance_type, endpoint_name=endpoint_name)

---------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Prepare input payload

We support JPEG, PNG, BMP, PDF, TIFF, HEIC, DOCX, PPTX, XLSX formats.

### C. Perform real-time inference

In [ ]:
import boto3
import json
from requests_toolbelt import MultipartEncoder

runtime_sm_client = boto3.client('runtime.sagemaker')

# Prepare multipart form data
m = MultipartEncoder(
    fields={
        'document': ('sample.pdf', open('../data/sample.pdf', 'rb'), 'application/pdf'),
        # 'document': ('solar.pdf', open('solar.pdf', 'rb'), 'application/pdf'), # for PDF files
        'model': 'document-parse',
        "ocr": 'auto', # force
        'coordinates': 'true',
        'output_formats': '["text", "html", "markdown"]',
        'base64_encoding': '["table"]',
        'chart_recognition': 'true',
    }
)

# Get the raw bytes of the multipart form data
body = m.to_string()

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=m.content_type,  # This will be 'multipart/form-data; boundary=...'
    Body=body
)

result = response["Body"].read()

# Print the result
print('output:', json.loads(result))

In [ ]:
res = json.loads(result)

In [ ]:
res.keys()

In [ ]:
res["content"].keys()

In [ ]:
res["elements"]#.keys()

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you can delete the endpoint and avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()